In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split

In [ ]:
mnist_train = MNIST(root='data', train=True, download=True, transform=ToTensor())
mnist_test = MNIST(root='data', train=False, download=True, transform=ToTensor())
X_train = mnist_train.data.view(-1, 784).float() / 255.0
y_train = mnist_train.targets
X_test = mnist_test.data.view(-1, 784).float() / 255.0
y_test = mnist_test.targets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
class Symbolic(nn.Module):
    def __init__(self):
        super(Symbolic, self).__init__()
        self.rule1 = nn.Linear(784, 10, bias=False)
        self.rule2 = nn.Linear(784, 10, bias=False)
        self.rule3 = nn.Linear(784, 10, bias=False)

    def forward(self, x):
        r1 = torch.sigmoid(self.rule1(x))
        r2 = torch.sigmoid(self.rule2(x))
        r3 = torch.sigmoid(self.rule3(x))
        x3 = (r1 + r2 + r3) / 3.0
        return x3

Define the neural part of the model

In [ ]:

class Neural(nn.Module):
    def __init__(self):
        super(Neural, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Define the combined model

In [ ]:

class NeuroSymbolic(nn.Module):
    def __init__(self):
        super(NeuroSymbolic, self).__init__()
        self.symbolic = Symbolic()
        self.neural = Neural()

    def forward(self, x):
        x1 = self.symbolic(x)
        x2 = self.neural(x)
        x3 = (x2 * x1) + ((1 - x2) * x)
        return x3

Initialize the model, loss function, and optimizer

In [ ]:

model = NeuroSymbolic()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Train the model

In [ ]:

for epoch in range(10):
    for i in range(0, len(X_train), 64):
        optimizer.zero_grad()
        outputs = model(X_train[i:i+64])
        loss = criterion(outputs, y_train[i:i+64])
        loss.backward()
        optimizer.step()

Print the training loss and accuracy every epoch

In [ ]:
# 
with torch.no_grad():
    outputs = model(X_train)
    _, predicted = torch.max(outputs.data, 1)
    train_acc = (predicted == y_train).sum().item() / len(y_train)
    train_loss = criterion(outputs, y_train)
    print('Epoch [%d/%d], Train Loss: %.4f, Train Accuracy: %.2f%%' % (epoch+1, 10, train_loss, train_acc*100))

    outputs = model(X_val)
    _, predicted = torch.max(outputs.data, 1)
    val_acc = (predicted == y_val).sum().item() / len(y_val)
    val_loss = criterion(outputs, y_val)
    print('Epoch [%d/%d], Validation Loss: %.4f, Validation Accuracy: %.2f%%' % (epoch+1, 10, val_loss, val_acc*100))